# 5/18, 6/1
発表担当：須藤

In [2]:
# Reproduce book environment
import random
random.seed(1234)

import logging
from pprint import pprint
from sys import stdout as STDOUT

# Write all output to a temporary directory
import atexit
import gc
import io
import os
import tempfile

TEST_DIR = tempfile.TemporaryDirectory()
atexit.register(TEST_DIR.cleanup)

# Make sure Windows processes exit cleanly
OLD_CWD = os.getcwd()
atexit.register(lambda: os.chdir(OLD_CWD))
os.chdir(TEST_DIR.name)

def close_open_files():
    everything = gc.get_objects()
    for obj in everything:
        if isinstance(obj, io.IOBase):
            obj.close()

atexit.register(close_open_files)

<function __main__.close_open_files()>

## 項目50 クラス属性に`__set_name__`で注釈を加える

メタクラスで実現されるもう1つの有用な機能は、クラスが定義された後で、実際に使われる前に、属性を修正したり、注釈を加えたりする能力である。この方式は、一般にディスクリプタを使って(「項目46 再利用可能な@property メソッドにディスクリプタを使う」 参照) 包含するクラス内においてそれがどのように使われるかを、さらに調べられるようにする。

例えば、顧客データベースのレコードを表す新しいクラスを定義したいとする。データベースの
テーブルの各カラムに対応する属性が必要である。そのために、属性とカラム名を連携するディスクリプタクラスを定義する。


In [3]:
# Example 1
class Field:
    def __init__(self, name):
        self.name = name
        self.internal_name = '_' + self.name

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

カラム名がFieldディスクリプタに格納されているため、すべてのインスタンスごとの状態は、み込み関数 setattr と getattr を使ってインスタンス辞書に保護フィールドとして直接保存できる。最初は、これは、メモリリークを避けるために weakref を使ったディスクリプタを構築するよりも、ずっと簡便に思える。

レコードを表現するクラスを定義するには、各クラス属性にカラム名を与える必要がある。


In [4]:
# Example 2
class Customer:
    # Class attributes
    first_name = Field('first_name')
    last_name = Field('last_name')
    prefix = Field('prefix')
    suffix = Field('suffix')


クラスの使い方は簡単で、Field ディスクリプタが期待通りにインスタンス辞書`__dict__`を修正していることがわかる。

In [5]:
# Example 3
cust = Customer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Euclid'
print(f'After:  {cust.first_name!r} {cust.__dict__}')

Before: '' {}
After:  'Euclid' {'_first_name': 'Euclid'}


しかし、クラス定義が冗長です。 ('field_name=') という代入文の左辺で、フィールド名を既に宣言している。右辺で (Field('first_name')) とフィールド名をFieldコンストラクタにも渡す
必要がなぜあるのだろうか。


In [6]:
# Example 4
class Customer:
    # 左辺と右辺が冗長
    first_name = Field('first_name')
    last_name = Field('last_name')
    prefix = Field('prefix')
    suffix = Field('suffix')


問題は Customer クラス定義における演算順序が、左から右へと読む順序とは反対になっていることです。最初に、FieldコンストラクタがField('first_name')で呼ばれます。それから、その戻り値がCustomer.first_nameに代入されます。Field インスタンスが前もって、どのクラス属性に代入されるかを知る方法はありません。

この冗長性をなくすために、メタクラスを使います。メタクラスはclass文に直接フックを掛けて
class 本体が終わるやいなや動作できます (働きの詳細は「項目48 サブクラスを`__init_subclass__`で検証する」参照)。この場合、フィールド名を何度も指定する代わりに、メタクラスを使ってField. nameとField.internal_nameとをディスクリプタで自動的に割り当てることができます。


In [7]:
# Example 5
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls

次に、メタクラスを使う基底クラスを定義します。 データベースのレコードを表すクラスはすべて メタクラスを使うように、このクラスを継承しなければなりません。

In [8]:
# Example 6
class DatabaseRow(metaclass=Meta):
    pass

メタクラスを使っても、フィールドディスクリプタはほとんど変わりません。唯一の相違点は、コンストラクタに渡す引数がもはや必要ないことです。代わりに、属性が上の `Meta.__new__` メソッドによって設定されます。


In [9]:
# Example 7
class Field:
    def __init__(self):
        # These will be assigned by the metaclass.
        self.name = None
        self.internal_name = None

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

メタクラスを使うことで、 新しい DatabaseRow 基底クラス、 新しいFieldディスクリプタ、そしてデータベースのレコードのためのクラス定義は、変更前にあった冗長性を解消しました。


In [10]:
# Example 8
class BetterCustomer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

新しいクラスの振る舞いは、 変更前と変わりません。

In [11]:
# Example 9
cust = BetterCustomer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Euler'
print(f'After:  {cust.first_name!r} {cust.__dict__}')


Before: '' {}
After:  'Euler' {'_first_name': 'Euler'}


この方式には、同じくDatabaseRow から継承されているものでないと、属性に Field クラスを使えないという問題点があります。DatabaseRow のサブクラスを忘れたり、クラス階層の他の構造的要求に従わないと、コードはうまくいきません。

In [12]:
# Example 10
try:
    class BrokenCustomer:
        first_name = Field()
        last_name = Field()
        prefix = Field()
        suffix = Field()
    
    cust = BrokenCustomer()
    cust.first_name = 'Mersenne'
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_27020\3518107673.py", line 10, in <cell line: 2>
    cust.first_name = 'Mersenne'
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_27020\2110137792.py", line 14, in __set__
    setattr(instance, self.internal_name, value)
TypeError: attribute name must be string, not 'NoneType'


この問題の解は、ディスクリプタに特殊メソッド `__set_name__` を使うことです。Python 3.6で導入されたこのメソッドは、含まれるクラスが定義されるときに、すべてのディスクリプタインスタンスについて呼び出されます。ディスクリプタインスタンスを保持する所有クラスとディスクリプタインスタンスが代入される属性名をパラメータとしてチェックします。次のコードでは、メタクラス全体の定 義は避けて、上で `Meta.__new__` メソッドが行っていることを `__set_name__` に移しています。


In [13]:
# Example 11
class Field:
    def __init__(self):
        self.name = None
        self.internal_name = None

    def __set_name__(self, owner, name):
        # Called on class creation for each descriptor
        self.name = name
        self.internal_name = '_' + name

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

今度は、スーパークラスから継承したり、メタクラスを使わなくても Fieldディスクリプタの便益
が得られます。


In [14]:
# Example 12
class FixedCustomer:
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

cust = FixedCustomer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Mersenne'
print(f'After:  {cust.first_name!r} {cust.__dict__}')

Before: '' {}
After:  'Mersenne' {'_first_name': 'Mersenne'}


### 覚えておくこと

- メタクラスは、クラスが完全に定義される前に、クラス属性の修正を可能にする。
- ディスクリプタとメタクラスとは、宣言的な振る舞いと実行時イントロスペクションのための強力なコンビである。
- ディスクリプタクラスで `__set_name__` を定義すると、取り囲むクラスとその属性名につ いての処理ができる。
- ディスクリプタがクラスインスタンスの階層において、 扱うデータを直接調べることで、メモリリークと組み込みモジュール weakref の両方を避けることができる。


## 項目51 合成可能なクラス拡張のためにはメタクラスではなくクラスデコレータを使う 

メタクラスで、クラス作成をさまざまな方法でカスタマイズ (「項目48 サブクラスを `__init_subclass__` で検証する」や「項目49 クラスの存在を`__init_subclass__`で登録する」 参照)できますが起こりうるあらゆる状況を扱うことはまだできません。
例えば、クラスの全メソッドについて、引数、戻り値、発生した例外を出力するヘルパーをデコレートしたいとします。 次のコードでは、デバッグデコレータを定義します (背景は「項目26 functools.wraps を使って関数デコレータを定義する」参照)。


In [15]:
# Example 1
from functools import wraps

def trace_func(func):
    if hasattr(func, 'tracing'):  # Only decorate once
        return func

    @wraps(func)
    def wrapper(*args, **kwargs):
        result = None
        try:
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            result = e
            raise
        finally:
            print(f'{func.__name__}({args!r}, {kwargs!r}) -> '
                  f'{result!r}')

    wrapper.tracing = True
    return wrapper

新たな dict サブクラスのさまざまな特殊メソッドに、このデコレータを適用できます (背景は「項目43 カスタムコンテナ型は collections.abc を継承する」参照)。

In [16]:
# Example 2
class TraceDict(dict):
    @trace_func
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @trace_func
    def __setitem__(self, *args, **kwargs):
        return super().__setitem__(*args, **kwargs)

    @trace_func
    def __getitem__(self, *args, **kwargs):
        return super().__getitem__(*args, **kwargs)

クラスのインスタンスとのやり取りで、これらのメソッドのデコレータの妥当性検証ができます。

In [17]:
# Example 3
trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except KeyError:
    pass  # Expected
else:
    assert False

__init__(({'hi': 1}, [('hi', 1)]), {}) -> None
__setitem__(({'hi': 1, 'there': 2}, 'there', 2), {}) -> None
__getitem__(({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__(({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')


このコードには、 @trace_func でデコレートしたいメソッドをすべて再定義しなければならないと いう問題点があります。これは、冗長なボイラープレート（使いまわされることを前提にした文言）で、読みにくくエラーになりやすいものです。 さらに、新たなメソッドを後で dict サブクラスに追加する場合には、TraceDict で定義しておかないとデコレートされません。

この問題を解決する方法の1つに、メタクラスを使ってクラスのメソッドをすべて自動的にデコレートする方法があります。 次のコードでは、この振る舞いを関数やメソッドをtrace_func デコレータの新たな型でラップして実装します。

In [18]:
# Example 4
import types

trace_types = (
    types.MethodType,
    types.FunctionType,
    types.BuiltinFunctionType,
    types.BuiltinMethodType,
    types.MethodDescriptorType,
    types.ClassMethodDescriptorType)

class TraceMeta(type):
    def __new__(meta, name, bases, class_dict):
        klass = super().__new__(meta, name, bases, class_dict)

        for key in dir(klass):
            value = getattr(klass, key)
            if isinstance(value, trace_types):
                wrapped = trace_func(value)
                setattr(klass, key, wrapped)

        return klass

メタクラス TraceMetaを使ってdict サブクラスを宣言し、 期待通り動作することを検証できます。

In [19]:
# Example 5
class TraceDict(dict, metaclass=TraceMeta):
    pass

trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except KeyError:
    pass  # Expected
else:
    assert False


__new__((<class '__main__.TraceDict'>, [('hi', 1)]), {}) -> {}
__getitem__(({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__(({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')


うまく動くだけでなく、 前の実装にはなかった__new__ の呼び出しも出力できています。 スーパークラスで既にメタクラスが指定されている場合、 TraceMetaを使うとどうなるでしょうか。

In [20]:
# Example 6
try:
    class OtherMeta(type):
        pass
    
    class SimpleDict(dict, metaclass=OtherMeta):
        pass
    
    class TraceDict(SimpleDict, metaclass=TraceMeta):
        pass
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_27020\3195519885.py", line 9, in <cell line: 2>
    class TraceDict(SimpleDict, metaclass=TraceMeta):
TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases


これは、TraceMetaがOther Metaを継承していないので、うまくいきません。理論的には、メタ クラス継承を使い、TraceMeta から Other Meta を継承してこの問題を解消できるはずです。

In [21]:
# Example 7
class TraceMeta(type):
    def __new__(meta, name, bases, class_dict):
        klass = type.__new__(meta, name, bases, class_dict)

        for key in dir(klass):
            value = getattr(klass, key)
            if isinstance(value, trace_types):
                wrapped = trace_func(value)
                setattr(klass, key, wrapped)

        return klass

class OtherMeta(TraceMeta):
    pass

class SimpleDict(dict, metaclass=OtherMeta):
    pass

class TraceDict(SimpleDict, metaclass=TraceMeta):
    pass

trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except KeyError:
    pass  # Expected
else:
    assert False

__init_subclass__((), {}) -> None
__new__((<class '__main__.TraceDict'>, [('hi', 1)]), {}) -> {}
__getitem__(({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__(({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')


しかし、修正不能なライブラリからのメタクラスであったり、 Trace Metaのようなユーティリティメタクラスを同時に複数使いたい場合には、これはうまくいきません。
メタクラス方式では、修正されるクラスにあまりに多くの制約があります。

この問題を回避するため、 Python にはクラスデコレータがあります。 クラスデコレータは、関数デコレータと同じように働きます。
@記号を関数名の前に付けて、クラス宣言の前に置きます。修正または再作成したクラスを、 関数が返します。


In [22]:
# Example 8
def my_class_decorator(klass):
    klass.extra_param = 'hello'
    return klass

@my_class_decorator
class MyClass:
    pass

print(MyClass)
print(MyClass.extra_param)

<class '__main__.MyClass'>
hello


上のTraceMeta.__new_ メソッドの主要部分を1つの関数に移して、 関数 trace_func をクラス のすべてのメソッドと関数に適用するクラスデコレータを実装できます。

In [23]:
# Example 9
def trace(klass):
    for key in dir(klass):
        value = getattr(klass, key)
        if isinstance(value, trace_types):
            wrapped = trace_func(value)
            setattr(klass, key, wrapped)
    return klass


このデコレータをdict サブクラスに適用して、 上のメタクラス方式で得られたのと同じ振る舞いが得られます。


In [24]:
# Example 10
@trace
class TraceDict(dict):
    pass

trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except KeyError:
    pass  # Expected
else:
    assert False

__new__((<class '__main__.TraceDict'>, [('hi', 1)]), {}) -> {}
__getitem__(({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__(({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')


クラスデコレータは、デコレートするクラスでメタクラスが既に定義済みの場合にもうまく動作します。

In [25]:
# Example 11
class OtherMeta(type):
    pass

@trace
class TraceDict(dict, metaclass=OtherMeta):
    pass

trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except KeyError:
    pass  # Expected
else:
    assert False

__new__((<class '__main__.TraceDict'>, [('hi', 1)]), {}) -> {}
__getitem__(({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__(({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')


クラスの拡張に、合成可能な方式を考えているなら、クラスデコレータが最良のツールです(役に 立つfunctools.total_ordering というクラスデコレータについては「項目73 優先度付きキュー でheapqの使い方を知っておく」参照)。

### 覚えておくこと


- クラスデコレータは class インスタンスを引数として、新たなクラスまたは元のクラスの修正バージョンを返す簡単な関数だ
- クラスデコレータは、最小の定型文ですべてのメソッドまたは属性を修正したい場合に役立つ
- メラクラスは、簡単に合成できないが、多くのクラスデコレータがクラスの拡張に問題なく使うことができる
